In [88]:
import os
import numpy as np
import matplotlib.pylab as plt
import cv2
from multiprocessing import Pool
from collections import defaultdict
import itertools

%matplotlib inline

In [13]:
POS_DIR = './data/good'
NEG_DIR = './data/bad'

In [10]:
pool = Pool(processes=26)

In [24]:
def pool_images(image_list):
    img_pool = defaultdict(dict)
    for img_name in image_list:
        img_ts = img_name.split('_')[0]
        img_cam = int(img_name.split('_')[1].split('.')[0])
        img_pool[img_ts][img_cam] = img_name
    return img_pool

In [81]:
def preprocess_img(img_dir, img_ts, image_names):
    images = {}
    for key, img_name in image_names.items():
        images[key] = cv2.imread(os.path.join(img_dir, img_name))
    return images

In [42]:
pos_images =  pool_images(os.listdir(POS_DIR))
neg_images = pool_images(os.listdir(NEG_DIR))
pos_ds = pool.map(preprocess_img, zip(itertools.repeat(POS_DIR, pos_images.keys(), pos_images.values()))


In [78]:
el0 = list(zip(itertools.repeat(POS_DIR), pos_images.keys(), pos_images.values()))[0]

In [75]:
el0

('./data/good',
 '1545332685.1158965',
 {1: '1545332685.1158965_1.jpg',
  2: '1545332685.1158965_2.jpg',
  0: '1545332685.1158965_0.jpg'})

In [83]:
tst_images = preprocess_img(*el0)

In [89]:
plt.imread(tst_images[0])

TypeError: Object does not appear to be a 8-bit string path or a Python file-like object